In [9]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mlb
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import math
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

#palette : {deep, muted, pastel, dark, bright, colorblind}
sns.set_color_codes("dark")

In [10]:
def convert_existing_pca_rs(input_rs, cookbook):
    """ This function takes existing PCA RS and outputs the new RS format."""
    
    req_set = set()

    pca_filter = input_rs['Req Source'] == 'PCA'

    input_rs = input_rs[pca_filter].copy()
    
    for i in input_rs['Req nr']:
        #print i
        req_set.add(i)
        
    
    cb_reqs = pd.Series(cookbook['Req nr'], copy=True)
    
    cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'H',value=r'')
    cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'V',value=r'')
    cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'a',value=r'')
    cookbook['Req nr'].replace(regex=True,inplace=True,to_replace=r'b',value=r'')
          
    compliancy_list = {}
    i = 0
    for x in input_rs['Req nr']:
        compliancy_list.update({x:input_rs.iloc[i]['Compliance Assessment']})
        i = i + 1
    
    feedback_list = {}
    i = 0
    for x in input_rs['Req nr']:
        feedback_list.update({x:input_rs.iloc[i]['Required additional controls/Compliance Feedback']})
        i = i + 1
    

    response_list = {}
    i = 0
    for x in input_rs['Req nr']:
        response_list.update({x:input_rs.iloc[i]['Response']})
        i = i + 1
        
    remref_list = {}
    i = 0
    for x in input_rs['Req nr']:
        remref_list.update({x:input_rs.iloc[i]['Remediation Project Ref']})
        i = i + 1
    
    res_comments_list = {}
    i = 0
    for x in input_rs['Req nr']:
        res_comments_list.update({x:input_rs.iloc[i]['Response Comments']})
        i = i + 1
    
    compliancy_series = []
    feedback_series = []
    response_series = []
    remref_series = []
    res_comments_series = []
    
    for i in range(0,104):
        if int(cookbook.iloc[i]['Req nr']) in req_set:
            compliancy_series.append(compliancy_list[cookbook.iloc[i]['Req nr']])
            feedback_series.append(feedback_list[cookbook.iloc[i]['Req nr']])
            response_series.append(response_list[cookbook.iloc[i]['Req nr']])
            remref_series.append(remref_list[cookbook.iloc[i]['Req nr']])
            res_comments_series.append(res_comments_list[cookbook.iloc[i]['Req nr']])
        else:
            compliancy_series.append(float('nan'))
            feedback_series.append(float('nan'))
            response_series.append(float('nan'))
            remref_series.append(float('nan'))
            res_comments_series.append(float('nan'))
            

    #print '###########################'
    #print cookbook.iloc[5]['Req nr']
    #print '###########################'
            
    cookbook['Compliance Assessment'] = compliancy_series
    cookbook['Required additional controls/Compliance Feedback'] = feedback_series
    cookbook['Response'] = response_series
    cookbook['Remediation Project Ref'] = remref_series
    cookbook['Response Comments'] = res_comments_series
    
    cookbook.sort_values(by=['Compliance Assessment','Req nr'],ascending=True, inplace=True)
    
    cookbook['Req nr'] = cb_reqs
    
    cookbook.rename(columns={'Cookbook RS Statements':'GDPR Requirement Specification',
                             'Privacy Requirement':'Requirement',
                             'People+Process Applicable':'People/Process Applicability',
                             'Technology  (& Design) Applicable':'Technology/Design Applicability'}, inplace=True)
    
    comliancy_filter = cookbook['Compliance Assessment'] != float('nan')
        
    return cookbook[comliancy_filter]

In [11]:
def convert_existing_sca_rs(input_rs, cookbook):
    """ This function takes existing SCA RS and outputs the new RS format."""
    
    req_set = set()

    sca_filter = input_rs['Req Source'] == 'SCA'

    input_rs = input_rs[sca_filter].copy()
    
    for i in input_rs['Req nr']:
        #print i
        req_set.add(i)
        
    
    cb_reqs = pd.Series(cookbook['RS Reference Number'], copy=True)
    
    compliancy_list = {}
    i = 0
    for x in input_rs['Req nr']:
        compliancy_list.update({str(x):input_rs.iloc[i]['Compliance Assessment']})
        i = i + 1
    
    feedback_list = {}
    i = 0
    for x in input_rs['Req nr']:
        feedback_list.update({str(x):input_rs.iloc[i]['Required additional controls/Compliance Feedback']})
        i = i + 1
    

    response_list = {}
    i = 0
    for x in input_rs['Req nr']:
        response_list.update({str(x):input_rs.iloc[i]['Response']})
        i = i + 1
        
    remref_list = {}
    i = 0
    for x in input_rs['Req nr']:
        remref_list.update({str(x):input_rs.iloc[i]['Remediation Project Ref']})
        i = i + 1
    
    res_comments_list = {}
    i = 0
    for x in input_rs['Req nr']:
        res_comments_list.update({str(x):input_rs.iloc[i]['Response Comments']})
        i = i + 1
    
    compliancy_series = []
    feedback_series = []
    response_series = []
    remref_series = []
    res_comments_series = []
    
    for i in range(0,15):
        if cookbook.iloc[i]['RS Reference Number'][10:-2] in req_set:
            compliancy_series.append(compliancy_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            feedback_series.append(feedback_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            response_series.append(response_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            remref_series.append(remref_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
            res_comments_series.append(res_comments_list[cookbook.iloc[i]['RS Reference Number'][10:-2]])
        else:
            compliancy_series.append(float('nan'))
            feedback_series.append(float('nan'))
            response_series.append(float('nan'))
            remref_series.append(float('nan'))
            res_comments_series.append(float('nan'))
            
    
    cookbook['Compliance Assessment'] = compliancy_series
    cookbook['Required additional controls/Compliance Feedback'] = feedback_series
    cookbook['Response'] = response_series
    cookbook['Remediation Project Ref'] = remref_series
    cookbook['Response Comments'] = res_comments_series
    
    cookbook.sort_values(by=['Compliance Assessment'],ascending=True, inplace=True)
    
    cookbook['Req nr'] = cb_reqs
    
    cookbook.rename(columns={'Cookbook RS Statements':'GDPR Requirement Specification',
                             'Privacy Requirement':'Requirement',
                             'People+Process Applicable':'People/Process Applicability',
                             'Technology  (& Design) Applicable':'Technology/Design Applicability'}, inplace=True)
    
    comliancy_filter = cookbook['Compliance Assessment'] != float('nan')
        
    return cookbook[comliancy_filter]

In [12]:
os.chdir('/home/andy/dev/notebooks/Transfer-RS-Feedback/old RS versions')

OUTPUT_BASE_PATH = '/home/andy/dev/notebooks/Transfer-RS-Feedback/old_output_rs/'
#OUTPUT_BASE_PATH = '/home/andy/dev/GDPR/converted_rs/'
OUT_FILENAME = 'outfile.xlsx'

COOKBOOK_FILE = '/home/andy/dev/GDPR/inputdata/PCA SCA Cookbook Mapping_2.2.xlsx'

cookbook = pd.read_excel(COOKBOOK_FILE, 
                         sheet_name='PCA',
                         skiprows=1)

cookbook_sca = pd.read_excel(COOKBOOK_FILE, 
                         sheet_name='SCA',
                         skiprows=1)

first = True
new_rs_sheets = {}

for filename in os.listdir(os.getcwd()):
    print filename
    output_writer = pd.ExcelWriter(OUTPUT_BASE_PATH + 'CONVERTED-PCA-' + filename[:14] + '.xlsx')

    df = pd.read_excel(filename, sheet_name=None)
    
    sheet_names = df.keys()
    sheets = sheet_names[4:-1]
    #print sheets

    for sh in sheets:
        #print sh
        if sh == u'PII Data Record': # this is a bug? Don't know why I need this.  For some reason the last sheet comes through.
            continue
        #new_rs_sheets.update({filename: dict({sh: convert_rs(df[sh], cookbook)})})
        out_df = convert_existing_pca_rs(df[sh], cookbook)
        
        out_df.to_excel(output_writer,sh)
        
    output_writer.save()

GDPR-RS-EPS027-V1.1 REVIEWED 14-03-2018.xlsx


KeyError: u'1'

In [ ]:
for filename in os.listdir(os.getcwd()):
    output_writer = pd.ExcelWriter(OUTPUT_BASE_PATH + 'CONVERTED-SCA-' + filename[:14] + '.xlsx')

    df = pd.read_excel(filename, sheet_name=None)
    
    sheet_names = df.keys()
    sheets = sheet_names[4:-1]
    #print sheets

    for sh in sheets:
        #print sh
        if sh == u'PII Data Record': # this is a bug? Don't know why I need this.  For some reason the last sheet comes through.
            continue
        #new_rs_sheets.update({filename: dict({sh: convert_rs(df[sh], cookbook)})})
        out_df = convert_existing_sca_rs(df[sh], cookbook_sca)
        out_df.to_excel(output_writer,sh)
        
    output_writer.save()